In [ ]:
#LISTA DE MODELOS
# ----------------------
# unsloth/llama-3.2-3b-bnb-4bit
# unsloth/llama-3.2-3b
# ----------------------
# unsloth/gemma-3-1b-it-bnb-4bit
# unsloth/gemma-3-1b-it
# ----------------------

In [4]:
# 001 -FINE-TUNING- INICIANDO O UNSLOTH #

import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
#pass

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-168_xuy9/unsloth_2e786517775444159276f5c184a9b260
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-168_xuy9/unsloth_2e786517775444159276f5c184a9b260
  Resolved https://github.com/unslothai/unsloth.git to commit d5b61a6bc6d546ca6afa9043f9c4b15713ecac62
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.6/180.6 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 103.1 MB/s eta 0:00:00

In [5]:

# 002 -FINE-TUNING- IMPORTS NECESSÁRIOS #

import os
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments, AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from datasets import Dataset
import xml.etree.ElementTree as ET
import pandas as pd
from peft import PeftModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [6]:
# 003 ACESSO AO GOOGLE DRIVE PARTICULAR #

from google.colab import drive
drive.mount('/content/drive')

pathDataSet = "/content/drive/MyDrive/06-ESTUDOS/FIAP/IA PARA DEVS/Fase 3 - Aulas/0000037_2.xml";

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# 004 -FINE-TUNING- MODELO ANTES-TREINO #

max_seq_length = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-3-1b-it-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True,
)

==((====))==  Unsloth 2025.12.7: Fast Gemma3 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: Gemma3 does not support SDPA - switching to fast eager.


model.safetensors:   0%|          | 0.00/965M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

In [8]:
# 005 -FINE-TUNING- AVALIANDO RESPOSTA ANTES DO TREINO #

def generate_text(text):
    inputs = tokenizer(text, return_tensors="pt").to("cuda:0")
    outputs = model.generate(**inputs, max_new_tokens=20)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

print("Antes do treino\n")
generate_text("<human>: What is (are) Rectal Cancer ?\n<bot>: ")

Antes do treino

<human>: What is (are) Rectal Cancer ?
<bot>: 
  
Rectal Cancer is a serious medical condition that occurs when the cells in the rectum,


In [9]:
# 006 -FINE-TUNING- CONFIGURAÇÃO LORA #

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = 16,
    lora_dropout = 0,  # Supports any, but = 0 is optimized
    bias = "none",     # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    max_seq_length = max_seq_length,
    use_rslora = False,   # Rank stabilized LoRA
    loftq_config = None,  # LoftQ
)

Unsloth: Making `model.base_model.model.model` require gradients


In [10]:
# 007 -FINE-TUNING- TRATAMENTO DATASET #

tree = ET.parse(pathDataSet)
root = tree.getroot()

data = []

focus = root.find("Focus")
focusText = focus.text

for qa in root.find("QAPairs").findall("QAPair"):
    pid = qa.get("pid")

    question_tag = qa.find("Question")
    question = question_tag.text.strip()
    qid = question_tag.get("qid")
    qtype = question_tag.get("qtype")

    answer = qa.find("Answer").text.strip()

    data.append({
        "pid": pid,
        "focus": focusText,
        "qid": qid,
        "qtype": qtype,
        "question": question,
        "answer": answer.replace("Key Points\n", "")
    })

df = pd.DataFrame(data)

from datasets import Dataset
dataset = Dataset.from_pandas(df)

def formatting_func(examples):
    texts = []
    for i in range(len(examples["question"])):
        question = examples["question"][i].strip()
        answer   = examples["answer"][i].strip()
        texts.append(f"<human>: {question}\n<bot>: {answer}")
    return texts

In [11]:
# 008 -FINE-TUNING- TREINO DO MODELO #

trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    formatting_func = formatting_func,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    tokenizer = tokenizer,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 60,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        output_dir = "outputs",
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Disable wandb
    ),
)

trainer.train()
trainer.save_model("lora_output")
model.save_pretrained("lora_output")
tokenizer.save_pretrained("lora_output")

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/8 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 8 | Num Epochs = 60 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 13,045,760 of 1,012,931,712 (1.29% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.490100
2,2.490200
3,2.485300
4,2.472100
5,2.447200
6,2.411800
7,2.369100
8,2.321200
9,2.269500
10,2.215100


('lora_output/tokenizer_config.json',
 'lora_output/special_tokens_map.json',
 'lora_output/chat_template.jinja',
 'lora_output/tokenizer.model',
 'lora_output/added_tokens.json',
 'lora_output/tokenizer.json')

In [17]:
# 009 -FINE-TUNING- AVALIANDO RESPOSTA DEPOIS DO TREINO #

print("Depois do treino\n")
generate_text("<human>: Who is at risk for Rectal Cancer?\n<bot>: ")

Depois do treino

<human>: Who is at risk for Rectal Cancer?
<bot>: 
     > The risks for colorectal cancer appear to be influenced by factors such as age, sex,


In [ ]:
# 010 -FINE-TUNING- ACESSO ARQUIVOS DO LORA #

#!ls lora_output # VISUALIZA ARQUIVOS
#!zip -r lora_output.zip lora_output # EXPORTA ARQUIVOS

In [18]:
# 001 -DEPLOY HUGGING FACE- ACESSO ARQUIVOS DO LORA #

!pip install -U huggingface_hub transformers accelerate safetensors

In [19]:
# 001 -DEPLOY HUGGING FACE- ACESSO ARQUIVOS DO LORA #

from huggingface_hub import login
from google.colab import userdata
login(token=userdata.get('meutoken'))

repositorioName = "dsantasantos/Modelo-gemma1b-tc03v001"

In [20]:
# 002 -DEPLOY HUGGING FACE- MERGE MODELO + LORA #

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

base_model = AutoModelForCausalLM.from_pretrained(
    "unsloth/gemma-3-1b-it",
    dtype=torch.float16,
    device_map="cpu",
)

tokenizer = AutoTokenizer.from_pretrained("unsloth/llama-3.2-1b")

model = PeftModel.from_pretrained(base_model, "lora_output")
model = model.merge_and_unload()

model.save_pretrained("modelo-final", safe_serialization=True)
tokenizer.save_pretrained("modelo-final")

config.json:   0%|          | 0.00/902 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

('modelo-final/tokenizer_config.json',
 'modelo-final/special_tokens_map.json',
 'modelo-final/tokenizer.json')

In [21]:
# 003 -DEPLOY HUGGING FACE- VISUALIZAR PASTA GERADA #

!ls modelo-final

config.json		model.safetensors	 tokenizer_config.json
generation_config.json	special_tokens_map.json  tokenizer.json


In [1]:
# 004 -DEPLOY HUGGING FACE- CRIAÇÃO + DEPLOY MODELO #

from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import create_repo

repositorioName = "dsantasantos/Modelo-gemma1b-tc03v001"

repo_id = repositorioName

model = AutoModelForCausalLM.from_pretrained("modelo-final")
tokenizer = AutoTokenizer.from_pretrained(
    "modelo-final",
    fix_mistral_regex=True
)

create_repo(repo_id, exist_ok=True)

model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ..._is7nxq/model.safetensors:   0%|          | 32.1kB / 4.00GB            

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mpc7h23vqx/tokenizer.json:  89%|########9 | 15.3MB / 17.2MB            

CommitInfo(commit_url='https://huggingface.co/dsantasantos/Modelo-gemma1b-tc03v001/commit/ae0f89d82807e53a8f67f937b78f46168755cb93', commit_message='Upload tokenizer', commit_description='', oid='ae0f89d82807e53a8f67f937b78f46168755cb93', pr_url=None, repo_url=RepoUrl('https://huggingface.co/dsantasantos/Modelo-gemma1b-tc03v001', endpoint='https://huggingface.co', repo_type='model', repo_id='dsantasantos/Modelo-gemma1b-tc03v001'), pr_revision=None, pr_num=None)

In [8]:
# 005 -DEPLOY HUGGING FACE- TESTE ENDPOINT #

from huggingface_hub import InferenceClient

client = InferenceClient(
    "https://wruo4nd68626ars4.us-east-1.aws.endpoints.huggingface.cloud",
    token=""
)

response = client.text_generation(
    "What is (are) Rectal Cancer ?",
    max_new_tokens=100
)

print(response)

What is (are) Rectal Cancer ?(name
